In [1]:
import networkx as nx
import matplotlib.pyplot as plt
from pyvis.network import Network

## Visualization

In [2]:
# Credit: https://gist.github.com/quadrismegistus/92a7fba479fc1e7d2661909d19d4ae7e

def visualize(networkx_graph, name):
    pyvis_graph = Network(height=800, width=800, notebook=True)
    for node,node_attrs in networkx_graph.nodes(data=True):
        pyvis_graph.add_node(node,**node_attrs)

    # for each edge and its attributes in the networkx graph
    for source,target,edge_attrs in networkx_graph.edges(data=True):
        # if value/width not specified directly, and weight is specified, set 'value' to 'weight'
        if not 'value' in edge_attrs and not 'width' in edge_attrs and 'weight' in edge_attrs:
            # place at key 'value' the weight of the edge
            edge_attrs['value']=edge_attrs['weight']
        # add the edge
        pyvis_graph.add_edge(source,target,**edge_attrs)

    return pyvis_graph.show('docs/' + name)

In [3]:
'''for weighted graph'''
def make_graph(coauth_tuples):
    G = nx.Graph()
    for coauthor in coauth_tuples:
        G.add_edge(coauthor[0], coauthor[1], weight=coauthor[2])
    return G

In [4]:
def filter(dataset, by_type, keyword):
    filtered_list = []
    for index in dataset:
        if keyword in dataset[index][by_type]:
            filtered_list.append(index)
    return filtered_list

In [5]:
'''build edge_list for specific distance for paper'''

def paper_add_edge(paper_dict, paper_id, distance, distance_count):
    edge_list = []
    if distance == distance_count:
        return
    distance_count += 1
    for node in paper_dict[paper_id]['%']:
        edge_list.append((paper_id, node))
        if paper_add_edge(paper_dict, node, distance, distance_count) is not None:
            edge_list += paper_add_edge(paper_dict, node, distance, distance_count)
    return edge_list

def paper_citation_edge_list(paper_dict, paper_id, distance):
    edge_list = []
    distance_count = 0
    edge_list+=paper_add_edge(paper_dict, paper_id, distance, distance_count)
    return (edge_list)

In [6]:
def author_add_edge(paper_dict, author, distance, distance_count):
    edge_list = []
    if distance == distance_count:
        return
    distance_count += 1
    paper_list = filter(paper_dict, '@', author)
    cite_author_list = []
    for paper in paper_list:
        if paper_dict[paper]['%'] is not None:
            for cite in paper_dict[paper]['%']:
                cite_author_list+=paper_dict[cite]['@']
    for cite_auth in cite_author_list:
        edge_list.append((author, cite_auth))
        if author_add_edge(paper_dict, cite_auth, distance, distance_count) is not None:
            edge_list += author_add_edge(paper_dict, cite_auth, distance, distance_count)      
    return edge_list  

def author_citation_edge_list(paper_dict, author, distance):
    edge_list = []
    distance_count = 0
    edge_list+=author_add_edge(paper_dict, author, distance, distance_count)
    return (edge_list)